# Task 3

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold

from sklearn.decomposition import TruncatedSVD
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.linear_model import RidgeClassifier

from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, f1_score
from sklearn.preprocessing import OneHotEncoder

from itertools import permutations 

In [2]:
test_data = pd.read_csv('test.csv') #These are just features names (48000)
train_data = pd.read_csv('train.csv') #Says if something is active or not (112000)

num_train_data = len(train_data)

In [3]:
train_feature = np.array(train_data['Sequence'])
train_output = np.array(train_data['Active'])
train_feature_splitted = []

In [4]:
for i, feature in enumerate(train_feature):
    train_feature_splitted.append([])
    for j, s in enumerate(list(feature)):
        train_feature_splitted[i].append(ord(s)*(j+1))

train_feature_splitted = np.array(train_feature_splitted)
train_feature_transform = np.copy(train_feature_splitted)

In [5]:
unique_feature = []
for j in range(4):
    unique_feature.append(np.unique(train_feature_splitted[:,j]))
    
unique_feature = np.array(unique_feature)

idx_list = []

for j in range(4):
    for i in range(20):
        idx_list.append(np.sum(train_output[train_feature_splitted[:,j]==unique_feature[j][i]]))
        
idx_list = np.array(idx_list)

sorted_idx = np.argsort(idx_list.flatten())
sorted_feature = unique_feature.flatten()[sorted_idx]

train_feature_sorted = np.zeros_like(train_feature_splitted)

for i, feature in enumerate(train_feature):
    for j, s in enumerate(list(feature)):
        train_feature_sorted[i,j] = sorted_feature[np.argwhere(unique_feature.flatten()==train_feature_splitted[i,j])]
        
train_feature_sorted = np.array(train_feature_sorted, dtype=float)

In [6]:
for i, j in permutations([0,1,2,3], 2):
    train_feature_mult = np.array([(train_feature_sorted[:,i]+1000)*train_feature_sorted[:,j]]).T
    train_feature_mult = train_feature_mult/np.max(train_feature_mult)
    train_feature_transform = np.concatenate((train_feature_transform, train_feature_mult), axis=1)

In [7]:
# for i, j, k in permutations([0,1,2,3], 3):
#     train_feature_mult = np.array([(train_feature_sorted[:,i]+1000)*
#                                    (train_feature_sorted[:,j]+300)*
#                                    train_feature_sorted[:,k]]).T
#     train_feature_mult = train_feature_mult/np.max(train_feature_mult)
#     train_feature_transform = np.concatenate((train_feature_transform, train_feature_mult), axis=1)

In [8]:
np.shape(train_feature_transform)

(112000, 16)

<112000x4848 sparse matrix of type '<class 'numpy.float64'>'
	with 1792000 stored elements in Compressed Sparse Row format>

In [20]:
train_feature_splitted = np.array(train_feature_transform, dtype=float)

ohe = OneHotEncoder(handle_unknown='ignore',sparse=False)

train_feature_ohe = ohe.fit_transform(train_feature_splitted)

print(np.shape(train_feature_ohe))

(112000, 4848)


In [10]:
# tsvd = TruncatedSVD(n_components=16, algorithm='arpack', random_state=None)
# train_feature_ohe_tsvd = tsvd.fit_transform(train_feature_ohe)

In [21]:
score_func = make_scorer(f1_score)

In [22]:
%%time
model = KNeighborsClassifier(n_neighbors=1, n_jobs=6)
scores = cross_val_score(model, train_feature_transform, train_output, cv=5, scoring=score_func)

print(scores)

[0.55038303 0.52637749 0.52513966 0.54577259 0.53068182]
Wall time: 6.89 s


In [13]:
%%time
model = DecisionTreeClassifier(criterion='entropy', random_state=None, class_weight='balanced')
scores = cross_val_score(model, train_feature_ohe, train_output, cv=5, scoring=score_func)

print(scores)

[0.79371955 0.80497029 0.78482385 0.7909292  0.79052235]
Wall time: 8.14 s


In [14]:
%%time
model = DecisionTreeClassifier(criterion='entropy', random_state=None, class_weight='balanced')
scores = cross_val_score(model, train_feature_transform, train_output, cv=5, scoring=score_func)

print(scores)

[0.73970944 0.73842453 0.71454656 0.76179104 0.76749703]
Wall time: 2.46 s


In [23]:
%%time
model = RandomForestClassifier(n_estimators=10, criterion='entropy', n_jobs=6)
scores = cross_val_score(model, train_feature_transform, train_output, cv=5, scoring=score_func)

print(scores)

[0.75549806 0.7503201  0.74141283 0.75848564 0.75976938]
Wall time: 1.51 s


In [26]:
%%time
model = RandomForestClassifier(n_estimators=100, criterion='entropy', n_jobs=6,
                               max_leaf_nodes=100,class_weight='balanced')
scores = cross_val_score(model, train_feature_ohe, train_output, cv=5, scoring=score_func)

print(scores)

[0.49891    0.48908028 0.49739982 0.48642124 0.49667827]
Wall time: 9min 57s


In [37]:
kf = KFold(n_splits=5, shuffle=False, random_state=None)
model = DecisionTreeClassifier(criterion='entropy', random_state=None, class_weight='balanced')
model2 = KNeighborsClassifier(n_neighbors=1, n_jobs=6)
#model = DecisionTreeRegressor(random_state=None)
#model = AdaBoostClassifier(n_estimators=10)

for train_idx, test_idx in kf.split(train_feature_splitted):
    
    model.fit(train_feature_ohe[train_idx], train_output[train_idx])
    model2.fit(train_feature_transform[train_idx], train_output[train_idx])
    
    prediction = model.predict(train_feature_ohe[test_idx])
    prediction2 = model2.predict(train_feature_transform[test_idx])
    print(f1_score(train_output[test_idx], np.logical_or(prediction, prediction2)))

KeyboardInterrupt: 

In [30]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5, 2), random_state=1,verbose=True)

In [40]:
clf.fit(train_feature_ohe,train_output)

KeyboardInterrupt: 

In [43]:
kf = KFold(n_splits=5, shuffle=False, random_state=None)
model = MLPClassifier(solver='lbfgs',hidden_layer_sizes=5,activation='tanh',verbose=True)
# model2 = KNeighborsClassifier(n_neighbors=1, n_jobs=6)
#model = DecisionTreeRegressor(random_state=None)
#model = AdaBoostClassifier(n_estimators=10)

for train_idx, test_idx in kf.split(train_feature_splitted):
    
    model.fit(train_feature_ohe[train_idx], train_output[train_idx])
#     model2.fit(train_feature_transform[train_idx], train_output[train_idx])
    
    prediction = model.predict(train_feature_ohe[test_idx])
#     prediction2 = model2.predict(train_feature_transform[test_idx])
#     print(f1_score(train_output[test_idx], np.logical_or(prediction, prediction2)))
print(f1_score(train_output[test_idx,prediction]))

IndexError: too many indices for array